In [1]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP
from pathlib import Path

In [2]:
import os 
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
dir_path = os.path.join(dir_path, 'Data')

def get_data(folder_path):
    X = []
    y = []
    dirs = os.listdir(folder_path)
    for path in tqdm(dirs):
        file_paths = os.listdir(os.path.join(folder_path, path))
        for file_path in tqdm(file_paths):
            with open(os.path.join(folder_path, path, file_path), 'r', encoding="utf-8") as f:
                lines = f.readlines()
                lines = ' '.join(lines)
                lines = gensim.utils.simple_preprocess(lines)
                #xử lí các kí tự đặc biệt
                lines = ' '.join(lines)
                lines = ViTokenizer.tokenize(lines)
                # tách các từ
                X.append(lines)
                y.append(path)

    return X, y

train_path = os.path.join(dir_path, 'D:\Private\DATA\DATA_LAW\TRAIN_FULL')
X_data, y_data = get_data(train_path)


100%|██████████| 6/6 [00:06<00:00,  1.10s/it]


In [3]:
test_path = os.path.join(dir_path, 'D:\Private\DATA\DATA_LAW\TEST_FULL')
X_test, y_test = get_data(test_path)

100%|██████████| 6/6 [00:02<00:00,  2.06it/s]


In [4]:
def save_to_file(data,file_path):
    with open(file_path,'w',encoding='utf-8') as f:
        for line in data:
            f.write(line + "\n")

save_to_file(X_data,'D://Private//DATA//Model_data//x_train_data.txt')
save_to_file(y_data, 'D://Private//DATA//Model_data//y_train_data.txt')


save_to_file(X_test,'D://Private//DATA//Model_data//x_test_data.txt')
save_to_file(y_test, 'D://Private//DATA//Model_data//y_test_data.txt')

In [5]:
import pickle

pickle.dump(X_data, open('D:/Private/DATA/Model_data/X_data.pkl', 'wb'))
pickle.dump(y_data, open('D:/Private/DATA/Model_data/y_data.pkl', 'wb'))

pickle.dump(X_test, open('D:/Private/DATA/Model_data/X_test.pkl', 'wb'))
pickle.dump(y_test, open('D:/Private/DATA/Model_data/y_test.pkl', 'wb'))

Feature Engineering

In [6]:
import pickle

X_data = pickle.load(open('D:/Private/DATA/Model_data/X_data.pkl', 'rb'))
y_data = pickle.load(open('D:/Private/DATA/Model_data/y_data.pkl', 'rb'))

X_test = pickle.load(open('D:/Private/DATA/Model_data/X_test.pkl', 'rb'))
y_test = pickle.load(open('D:/Private/DATA/Model_data/y_test.pkl', 'rb'))


WAY 1: Count Vectors as features

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_data)

# transform the training and validation data using count vectorizer object
X_data_count = count_vect.transform(X_data)
X_test_count = count_vect.transform(X_test)


In [26]:
print(X_test_count)

  (0, 79)	1
  (0, 191)	1
  (0, 252)	1
  (0, 280)	1
  (0, 294)	1
  (0, 365)	1
  (0, 449)	2
  (0, 452)	1
  (0, 453)	1
  (0, 548)	1
  (0, 634)	2
  (0, 699)	1
  (0, 887)	1
  (0, 993)	1
  (0, 1329)	1
  (0, 1572)	1
  (0, 1685)	1
  (0, 1730)	1
  (0, 1808)	2
  (0, 1810)	1
  (0, 1812)	1
  (0, 1886)	1
  (0, 1922)	2
  (0, 1941)	1
  (0, 2038)	1
  :	:
  (721, 1606)	11
  (721, 1624)	5
  (721, 1693)	1
  (721, 1730)	2
  (721, 1941)	2
  (721, 2038)	1
  (721, 2172)	7
  (721, 2267)	5
  (721, 2268)	2
  (721, 2305)	1
  (721, 2447)	8
  (721, 2564)	6
  (721, 2628)	2
  (721, 2692)	3
  (721, 2742)	1
  (721, 2774)	3
  (721, 2875)	1
  (721, 2882)	1
  (721, 2885)	7
  (721, 3059)	4
  (721, 3071)	1
  (721, 3079)	3
  (721, 3173)	3
  (721, 3187)	3
  (721, 3285)	1


Tf-Idf Vectors as Features

Word level

In [8]:
# word level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(X_data) # learn vocabulary and idf from training set
X_data_tfidf =  tfidf_vect.transform(X_data)
# assume that we don't have test set before
X_test_tfidf =  tfidf_vect.transform(X_test)


Ngram_level

In [9]:
# ngram level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram.fit(X_data)
X_data_tfidf_ngram =  tfidf_vect_ngram.transform(X_data)
# assume that we don't have test set before
X_test_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)


 SVD (singular value decomposition)

Word Level

In [11]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(X_data_tfidf)


X_data_tfidf_svd = svd.transform(X_data_tfidf)
X_test_tfidf_svd = svd.transform(X_test_tfidf)


N-gram Level

In [12]:
svd_ngram = TruncatedSVD(n_components=300, random_state=42)
svd_ngram.fit(X_data_tfidf_ngram)

X_data_tfidf_ngram_svd = svd_ngram.transform(X_data_tfidf_ngram)
X_test_tfidf_ngram_svd = svd_ngram.transform(X_test_tfidf_ngram)


Word Embeddings

In [14]:

from gensim.models import KeyedVectors 
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
word2vec_model_path = os.path.join(dir_path, "vi.vec")

w2v = KeyedVectors.load_word2vec_format(word2vec_model_path)
vocab = w2v.key_to_index
wv = w2v

def get_word2vec_data(X):
    word2vec_data = []
    for x in X:
        sentence = []
        for word in x.split(" "):
            if word in vocab:
                sentence.append(wv[word])

        word2vec_data.append(sentence)

    return word2vec_data

X_data_w2v = get_word2vec_data(X_data)
X_test_w2v = get_word2vec_data(X_test)


TRAIN MODEL

In [15]:
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [29]:
def train_model(classifier, X_data, y_data, X_test, y_test, is_neuralnet=False, n_epochs=3):       
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=42)
    
    if is_neuralnet:
        classifier.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=512)
        
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(X_train, y_train)
    
        train_predictions = classifier.predict(X_train)
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)

        
    print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))
    print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))


In [17]:
X_data, y_data, X_test, y_test = X_data_tfidf, y_data, X_test_tfidf, y_test
classifier = LogisticRegression()
model = train_model(classifier,X_data, y_data, X_test, y_test)

X_train shape: (1508, 3345)
y_train shape: (1508,)
X_test shape: (722, 3345)
y_test shape: (722,)
Validation accuracy:  0.8452380952380952
Test accuracy:  0.8601108033240997


In [18]:
input_file = 'D:\\Private\\DATA\Model\\bbbb.txt'

with open(input_file,'r',encoding='utf-8') as f:
    text = f.read()
text = ViTokenizer.tokenize(text)
text_to_vect = tfidf_vect.transform([text])
classifier.predict(text_to_vect)


array(['dau_thau'], dtype='<U8')

In [19]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import models, optimizers

In [20]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, LSTM, Dense
from tensorflow.keras import models, optimizers

def create_lstm_model():
    input_layer = Input(shape=(300,))
    
    # Reshape dữ liệu thành (10, 30)
    layer = Reshape((10, 30))(input_layer)
    
    # Thêm lớp LSTM
    layer = LSTM(128, return_sequences=False)(layer)
    
    # Thêm các lớp Dense
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)
    
    # Lớp đầu ra
    output_layer = Dense(10, activation='softmax')(layer)
    
    # Tạo mô hình
    classifier = models.Model(inputs=input_layer, outputs=output_layer)
    
    # Biên dịch mô hình
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier


In [30]:
X_data, y_data, X_test, y_test = X_data_w2v, y_data, X_test_w2v, y_test

model = create_lstm_model()
classifier = model
train_model(classifier,X_data, y_data, X_test, y_test)